In [1]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz

In [2]:
caminho_top300Abras = 'Ranking_ABRAS_2025_TOP300 .xlsx'
caminho_clientes = 'CliemtesVitao.xlsx'

In [3]:
df_Top300 = pd.read_excel(caminho_top300Abras)
df_clientes = pd.read_excel(caminho_clientes)

In [4]:
df_Top300.head()

,CNPJ Raiz,Data,Posição 2025,Posição 2024,Razão Social,Sede,Faturamento (R$)
0,NaN,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000
1,NaN,2025-01-01,2,2,ASSAÍ ATACADISTA,SP,80570000000
2,NaN,2025-01-01,3,3,MATEUS SUPERMERCADOS S.A.,MA,36385706000
3,NaN,2025-01-01,4,5,SUPERMERCADOS BH COMÉRCIO DE ALIMENTOS S.A.,MG,21278845531
4,NaN,2025-01-01,5,4,GPA,SP,20047800000


In [5]:
df_clientes.head()

,sap_cod_cliente,cnpj_raiz,razao_social
0,2000000126,128288810,GL DISTRIBUICAO COMERCIO DE PRODUTOS ALIMENTIC...
1,2000000024,132728920,B&B DISTRIBUIDORA COMERCIAL BRASIL LTDA - EPP
2,2000000027,115250480,TRANSAFE TRANSPORTES LTDA
3,2000000029,113111010,CAMOL DISTRIBUIDORA LTDA - ME
4,2000000035,102813450,JBL TRANSPORTES LTDA - EPP


In [6]:
print("COLUNAS DO TOP 300:")
print(df_Top300.columns.tolist())
print("\n" + "="*60 + "\n")

print("COLUNAS DOS CLIENTES:")
print(df_clientes.columns.tolist())

COLUNAS DO TOP 300:
['CNPJ Raiz', 'Data', 'Posição 2025', 'Posição 2024', 'Razão Social', 'Sede', 'Faturamento (R$)']


COLUNAS DOS CLIENTES:
['sap_cod_cliente', 'cnpj_raiz', 'razao_social']


In [7]:
coluna_nome_top300 = 'Razão Social'
coluna_cnpj_top300 = 'CNPJ Raiz'

coluna_nome_clientes = 'razao_social'
coluna_cnpj_clientes = 'cnpj_raiz'

print("Configuração das colunas:")
print(f"   Top 300 - Nome: '{coluna_nome_top300}'")
print(f"   Top 300 - CNPJ: '{coluna_cnpj_top300}'")
print(f"   Clientes - Nome: '{coluna_nome_clientes}'")
print(f"   Clientes - CNPJ: '{coluna_cnpj_clientes}'")

Configuração das colunas:
   Top 300 - Nome: 'Razão Social'
   Top 300 - CNPJ: 'CNPJ Raiz'
   Clientes - Nome: 'razao_social'
   Clientes - CNPJ: 'cnpj_raiz'


In [8]:
def limpar_nome(nome):
    
    if pd.isna(nome):
        return ""
    
    nome = str(nome).upper().strip()
    

    termos_remover = [
        ' LTDA.', ' LTDA', ' LTD.', ' LTD',
        ' S/A', ' S.A.', ' S.A', ' SA',
        ' EIRELI', ' ME', ' EPP', ' CIA.', ' CIA',
        ' COMERCIO', ' COMÉRCIO', ' VAREJISTA',
        ' E INDUSTRIA', ' E INDÚSTRIA',
        ' MAGAZINE', ' MAGAZINES',
        ' SUPERMERCADO', ' SUPERMERCADOS',
        ' HIPERMERCADO', ' HIPERMERCADOS',
        ' DISTRIBUIDORA', ' DISTRIBUIDOR',
        ' - EPP', ' - ME', ' - EIRELI'
    ]
    

    for termo in termos_remover:
        nome = nome.replace(termo, '')
    
    caracteres_remover = ['.', ',', '/', '\\', '-', '(', ')']
    for char in caracteres_remover:
        nome = nome.replace(char, ' ')
    
    nome = ' '.join(nome.split())
    
    return nome

print("TESTANDO A FUNÇÃO CORRIGIDA:\n")

exemplos_teste = [
    'CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.',
    'ASSAÍ ATACADISTA',
    'MATEUS SUPERMERCADOS S.A.',
    'GPA',
    'B&B DISTRIBUIDORA COMERCIAL BRASIL LTDA - EPP'
]

for exemplo in exemplos_teste:
    limpo = limpar_nome(exemplo)
    print(f"Original: {exemplo:55} → Limpo: {limpo}")

TESTANDO A FUNÇÃO CORRIGIDA:

Original: CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.                    → Limpo: CARREFOUR
Original: ASSAÍ ATACADISTA                                        → Limpo: ASSAÍ ATACADISTA
Original: MATEUS SUPERMERCADOS S.A.                               → Limpo: MATEUSS
Original: GPA                                                     → Limpo: GPA
Original: B&B DISTRIBUIDORA COMERCIAL BRASIL LTDA - EPP           → Limpo: B&B COMERCIAL BRASIL


In [9]:
df_Top300['razao_social_limpa'] = df_Top300[coluna_nome_top300].apply(limpar_nome)
df_clientes['razao_social_limpa'] = df_clientes[coluna_nome_clientes].apply(limpar_nome)

print("Colunas criadas com sucesso!")

Colunas criadas com sucesso!


In [10]:
df_Top300.head(10)

,CNPJ Raiz,Data,Posição 2025,Posição 2024,Razão Social,Sede,Faturamento (R$),razao_social_limpa
0,NaN,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR
1,NaN,2025-01-01,2,2,ASSAÍ ATACADISTA,SP,80570000000,ASSAÍ ATACADISTA
2,NaN,2025-01-01,3,3,MATEUS SUPERMERCADOS S.A.,MA,36385706000,MATEUSS
3,NaN,2025-01-01,4,5,SUPERMERCADOS BH COMÉRCIO DE ALIMENTOS S.A.,MG,21278845531,SUPERMERCADOS BH DE ALIMENTOS
4,NaN,2025-01-01,5,4,GPA,SP,20047800000,GPA
5,NaN,2025-01-01,6,6,IRMÃOS MUFFATO S.A.,PR,17433234858,IRMÃOS MUFFATO
6,NaN,2025-01-01,7,7,GRUPO PEREIRA,SP,15326536185,GRUPO PEREIRA
7,NaN,2025-01-01,8,9,MART MINAS ATACADO E VAREJO & DOM ATACADISTA,MG,11432757233,MART MINAS ATACADO E VAREJO & DOM ATACADISTA
8,NaN,2025-01-01,9,8,CENCOSUD BRASIL COMERCIAL LTDA.,SP,11235202790,CENCOSUD BRASIL COMERCIAL
9,NaN,2025-01-01,10,10,KOCH HIPERMERCADO S.A.,SC,10340540000,KOCH


In [11]:
df_clientes.head(10)

,sap_cod_cliente,cnpj_raiz,razao_social,razao_social_limpa
0,2000000126,128288810,GL DISTRIBUICAO COMERCIO DE PRODUTOS ALIMENTIC...,GL DISTRIBUICAO DE PRODUTOS ALIMENTICIOS
1,2000000024,132728920,B&B DISTRIBUIDORA COMERCIAL BRASIL LTDA - EPP,B&B COMERCIAL BRASIL
2,2000000027,115250480,TRANSAFE TRANSPORTES LTDA,TRANSAFE TRANSPORTES
3,2000000029,113111010,CAMOL DISTRIBUIDORA LTDA - ME,CAMOL
4,2000000035,102813450,JBL TRANSPORTES LTDA - EPP,JBL TRANSPORTES
5,2000000042,78052690,F. AGUIAR TRANSPORTES LTDA - ME,F AGUIAR TRANSPORTES
6,2000000047,54288230,LOGISTICA DOIS IRMAOS LTDA - ME,LOGISTICA DOIS IRMAOS
7,2000000051,35112980,MVM TRANSPORTES EIRELI,MVM TRANSPORTES
8,2000000055,18507880,IDEAL BRASIL LOGISTICA E TRANSP. DE CARGAS LTDA,IDEAL BRASIL LOGISTICA E TRANSP DE CARGAS
9,2000000057,10399250,TRANSHIZZA TRANSPORTES DE CARGAS E ENCOMENDAS ...,TRANSHIZZA TRANSPORTES DE CARGAS E ENCOMENDAS


In [12]:
df_Top300[coluna_cnpj_top300] = (
    df_Top300[coluna_cnpj_top300]
        .astype(str)
        .str.replace(r'\D', '', regex=True)   # remove tudo que não é número
        .str.replace(' ', '', regex=False)   # remove espaços (segurança extra)
)

df_clientes[coluna_cnpj_clientes] = (
    df_clientes[coluna_cnpj_clientes]
        .astype(str)
        .str.replace(r'\D', '', regex=True)
        .str.replace(' ', '', regex=False)
)

In [13]:
nomes_clientes = df_clientes['razao_social_limpa'].dropna().unique().tolist()

In [14]:
def nome_aproximado(nome, lista_nomes,score_min=80):
    if pd.isna(nome) or nome == "":
        return pd.Series([None, 0])

    match = process.extractOne(
        nome,
        lista_nomes,
        scorer=fuzz.token_sort_ratio
    )

    if match and match[1] >= score_min:
        return pd.Series([match[0], match[1]])
    else:
        return pd.Series([None, match[1] if match else 0])

In [15]:
df_Top300[['nome_cliente_match', 'score_match']] = (
    df_Top300['razao_social_limpa']
    .apply(lambda x:nome_aproximado(x, nomes_clientes))
)

In [16]:
df_Top300 = df_Top300.merge(
    df_clientes[['razao_social_limpa', coluna_cnpj_clientes]],
    left_on='nome_cliente_match',
    right_on='razao_social_limpa',
    how='left'
)

In [17]:
df_Top300.head()

,CNPJ Raiz,Data,Posição 2025,Posição 2024,Razão Social,Sede,Faturamento (R$),razao_social_limpa_x,nome_cliente_match,score_match,razao_social_limpa_y,cnpj_raiz
0,,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR,CARREFOUR,100.0,CARREFOUR,455439150
1,,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR,CARREFOUR,100.0,CARREFOUR,455439150
2,,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR,CARREFOUR,100.0,CARREFOUR,455439150
3,,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR,CARREFOUR,100.0,CARREFOUR,455439150
4,,2025-01-01,1,1,CARREFOUR COMÉRCIO E INDÚSTRIA LTDA.,SP,120594000000,CARREFOUR,CARREFOUR,100.0,CARREFOUR,455439150


In [18]:
df_Top300.to_excel('Top300Formatado(2).xlsx', index=False)